In [1]:
from selenium import webdriver
import requests
import time
import pandas as pd
from IPython.display import clear_output

In [2]:
# 웹드라이브 실행
driver = webdriver.Chrome('chromedriver')

In [3]:
# 다음 영화 사이트를 들어간다.
site = 'https://movie.daum.net/premovie/theater'
time.sleep(1)
driver.get(site)

In [4]:
# 스크롤 한다.
for idx in range(10) :
    time.sleep(1)
    script = 'window.scrollTo(0, document.body.scrollHeight)'
    driver.execute_script(script)

In [5]:
# 영화 전체 목록을 가져온다.
a1 = driver.find_element_by_css_selector('#mainContent > div > div.box_movie > ol')
a1

<selenium.webdriver.remote.webelement.WebElement (session="eeeb3fd28f0219a214c56123deb53e5a", element="a508a4a0-0bd5-46d0-af9d-d2528d12b6c4")>

In [6]:
# 영화 전체 목록에서 li 태그들을 가져온다.
li_list = a1.find_elements_by_css_selector('li')
li_list

[<selenium.webdriver.remote.webelement.WebElement (session="eeeb3fd28f0219a214c56123deb53e5a", element="41ed0ca7-38c7-439e-a655-c80d8d44c3aa")>,
 <selenium.webdriver.remote.webelement.WebElement (session="eeeb3fd28f0219a214c56123deb53e5a", element="28dce678-b2f4-4e8a-a8d7-c55970995543")>,
 <selenium.webdriver.remote.webelement.WebElement (session="eeeb3fd28f0219a214c56123deb53e5a", element="8e76dedc-bad5-4e25-bf0b-f8c1af5a6474")>,
 <selenium.webdriver.remote.webelement.WebElement (session="eeeb3fd28f0219a214c56123deb53e5a", element="b6112882-1461-4d57-8057-4e1b0ffb95f5")>,
 <selenium.webdriver.remote.webelement.WebElement (session="eeeb3fd28f0219a214c56123deb53e5a", element="2b37caba-55e8-4506-bef4-e6d3bb76592d")>,
 <selenium.webdriver.remote.webelement.WebElement (session="eeeb3fd28f0219a214c56123deb53e5a", element="618045b3-d71e-4e47-9ab4-6738b53cfd4d")>,
 <selenium.webdriver.remote.webelement.WebElement (session="eeeb3fd28f0219a214c56123deb53e5a", element="f287109a-29dd-4959-9cf2-35

In [7]:
# 데이터를 담을 딕셔너리
data_dict = {
    '제목' : [],
    '예매율' : [],
    '평점' : [],
    '개봉일' : []
}

# 영화의 수 만큼 반복한다.
for movie_tag in li_list :
    # 영화 제목을 가져온다.
    title_tag = movie_tag.find_element_by_css_selector('div > div.thumb_cont > strong > a')
    title = title_tag.text.strip()
    
    # 평점
    rating_tag = movie_tag.find_element_by_css_selector('div > div.thumb_cont > span.txt_append > span:nth-child(1) > span')
    rating = rating_tag.text.strip()
    
    # 예매율
    ticket_tag = movie_tag.find_element_by_css_selector('div > div.thumb_cont > span.txt_append > span:nth-child(2) > span')
    ticket = ticket_tag.text.strip()
    
    # 개봉일
    open_date_tag = movie_tag.find_element_by_css_selector('div > div.thumb_cont > span.txt_info > span')
    open_date = open_date_tag.text.strip()
    
    # 영화 포스터
    poster_tag = movie_tag.find_element_by_css_selector('div > div.thumb_item > div.poster_movie > img')
    # src 속성의 값을 가져온다
    src_attr = poster_tag.get_attribute('src')
    # print(src_attr)
    
    # 영화의 제목을 파일명으로 사용할 것이기 때문에 
    # os에서 거부하는 파일명을 정제한다.
    
    char_list = ['\\', '/', ':', '*', '?', '"', '<', '>', '|']
    
    file_name = title
    
    for c1 in char_list :
        file_name = file_name.replace(c1, ' ')
        
    # 이미지 데이터를 내려받는다.
    img_res = requests.get(src_attr)
    
    # 저장한다.
    with open(f'poster/{file_name}.jpg', 'wb') as fp :
        fp.write(img_res.content)
    
    # print(title)
    # print(rating)
    # print(ticket[:-1])
    # print(open_date)
    # print('-----------------------')
    
    # 데이터를 담는다.
    data_dict['제목'].append(title)
    data_dict['예매율'].append(ticket[:-1])
    data_dict['평점'].append(rating)
    data_dict['개봉일'].append(open_date)
    
# 데이터 프레임 생성
df1 = pd.DataFrame(data_dict)
# df1
# 저장
df1.to_csv('daum_movie.csv', encoding='utf-8-sig', index=False)
print('저장완료')

PermissionError: [Errno 13] Permission denied: 'daum_movie.csv'